<a href="https://colab.research.google.com/github/wiatrak2/BScThesis/blob/master/BackwardFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('BScThesis/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 591.8MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62826000 @  0x7fda7e9502a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 21.1MB/s 
    100% |████████████████████████████████| 2.0MB 6.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [111]:
!git clone https://github.com/wiatrak2/BScThesis

Cloning into 'BScThesis'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 364 (delta 5), reused 9 (delta 3), pack-reused 350
Receiving objects: 100% (364/364), 56.72 MiB | 10.07 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [0]:
from importlib import reload 

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import torchvision
import torchvision.transforms as transforms

In [0]:
import matplotlib.pyplot as plt
import pickle
import sympy
from collections import defaultdict, namedtuple
import copy

In [0]:
from mnist_m import Mnist_M
from random_dataset import RandomDataset
import datasets_merge
import models
import svhn_models
import trainer
import domain_trainer
from test_model import test_model
import plots
import test
import plotly_plots

In [0]:
import backward_functions

In [0]:
import plotly
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

In [0]:
def convertMnist(img):
  img = img.squeeze().numpy()
  return torch.from_numpy(np.tile(img, (3,1,1)))

In [0]:
batch_size=128
num_workers=1

In [12]:
mnist_transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081)),
                           transforms.Lambda(convertMnist)
                       ])

mnist_train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True,
                                     transform=mnist_transform)
mnist_train_loader = torch.utils.data.DataLoader(mnist_train_set, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

mnist_test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True,
                                     transform=mnist_transform)
mnist_test_loader = torch.utils.data.DataLoader(mnist_test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)



Processing...
Done!


In [0]:
mnist_m_transform = transforms.Compose([
                           transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081)),
                       ])
mnist_m_train_set = Mnist_M('drive/My Drive/BScThesis/keras_mnistm.pkl',
                        transform=mnist_m_transform)
mnist_m_test_set = Mnist_M('drive/My Drive/BScThesis/keras_mnistm.pkl',train=False,
                     transform=mnist_m_transform)
mnist_m_train_loader = torch.utils.data.DataLoader(mnist_m_train_set, 
                                                   batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)
mnist_m_test_loader = torch.utils.data.DataLoader(mnist_m_test_set,
                                                  batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

In [0]:
merge_train_set = datasets_merge.Merge_Datasets([mnist_train_set, mnist_m_train_set], get_labels=False)
merge_test_set = datasets_merge.Merge_Datasets([mnist_test_set, mnist_m_test_set], get_labels=False)
merge_train_loader = torch.utils.data.DataLoader(merge_train_set, 
                                                   batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)
merge_test_loader = torch.utils.data.DataLoader(merge_test_set, 
                                                   batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

In [0]:
log_interval = 100

In [0]:
use_cuda = True
use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [0]:
lr = 0.01
momentum=0.9

In [0]:
Models = namedtuple('Models', ['model_f', 'model_c', 'model_d'])
Optims = namedtuple('Optims', ['optim_f', 'optim_c', 'optim_d'])
Criterions = namedtuple('Criterions', ['criterion', 'criterion_domain'])
Loaders = namedtuple('Loaders', ['train_loader', 'train_loader_domain',
                                'source_test_loader', 'target_test_loader',
                                'merged_test_loader'])

In [0]:
loaders = Loaders(mnist_train_loader, mnist_m_train_loader,
                 mnist_test_loader, mnist_m_test_loader, merge_test_loader)

In [0]:
criterion = F.nll_loss
criterion_domain = F.nll_loss

In [0]:
criterions = Criterions(criterion, criterion_domain)

In [0]:
epochs=10

# Testing functions

In [0]:
def test_grad_function(grad_function, epochs=4):
  model_f = models.MnistFeatureExtractor().to(device)
  model_c = models.MnistClassPredictor().to(device)
  model_d = models.MnistDomain(grad_func = grad_function).to(device)
  
  optim_f = optim.SGD(model_f.parameters(), lr=lr, momentum=momentum, nesterov=True)
  optim_c = optim.SGD(model_c.parameters(), lr=lr, momentum=momentum, nesterov=True)
  optim_d = optim.SGD(model_d.parameters(), lr=lr, momentum=momentum, nesterov=True)
  
  train_models = Models(model_f, model_c, model_d)
  optims = Optims(optim_f, optim_c, optim_d)
  
  model_trainer = trainer.Trainer(train_models, optims, criterions, device,
                                train_domain=True, tune_lr=True)
  
  test_history = defaultdict(lambda:[])
  model_trainer.train(4, loaders, test_history=test_history)

In [0]:
grad_log = []
def grlog(x, lambd):
  return backward_functions.GRLog(lambd, grad_log)(x)

In [0]:
grad_log = []
def grinv(x, lambd):
  return backward_functions.GRInv(-1, lambd, grad_log)(x)

In [0]:
test_grad_function(grlog)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.302571, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.632978, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.591048, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.404582, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.264745, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 1.2978, Accuracy: 6271/10000 (63%)
Source Domain Test set: Average loss: 0.1211, Accuracy: 9614/10000 (96%)
Domains predictor:  Accuracy: 14577/20000 (73%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.295873, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.283074, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.267564, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.297445, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.162265, lr: 0.00273

In [68]:
test_grad_function(grinv)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.318720, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.810656, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.549511, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.457427, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.222553, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 1.7708, Accuracy: 6613/10000 (66%)
Source Domain Test set: Average loss: 0.1464, Accuracy: 9558/10000 (96%)
Domains predictor:  Accuracy: 10350/20000 (52%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.371020, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.264524, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.335705, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.398240, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.152408, lr: 0.00273

In [0]:
grad_log = []
def grinv2(x, lambd):
  return backward_functions.GRInv(1., lambd, grad_log)(x)

In [95]:
test_grad_function(grinv2)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.327919, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.745220, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.533236, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.456913, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.288192, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 2.3159, Accuracy: 3049/10000 (30%)
Source Domain Test set: Average loss: 0.0995, Accuracy: 9698/10000 (97%)
Domains predictor:  Accuracy: 19797/20000 (99%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.233389, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.372094, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.303700, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.326173, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.291125, lr: 0.00273

In [0]:
grad_log = []
def grandmult(x, lambd):
  return backward_functions.GRandMult(1, lambd, grad_log)(x)

In [97]:
test_grad_function(grandmult)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.356206, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.634399, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.509887, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.173120, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.351211, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 1.9298, Accuracy: 3438/10000 (34%)
Source Domain Test set: Average loss: 0.1113, Accuracy: 9651/10000 (97%)
Domains predictor:  Accuracy: 19679/20000 (98%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.223685, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.170962, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.182587, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.096928, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.161787, lr: 0.00273

In [0]:
grad_log = []
def grandmult(x, lambd):
  return backward_functions.GRandMult(1e-1, lambd, grad_log)(x)

In [92]:
test_grad_function(grandmult)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.312164, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.551612, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.499983, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.353211, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.277616, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 1.5431, Accuracy: 4493/10000 (45%)
Source Domain Test set: Average loss: 0.1045, Accuracy: 9670/10000 (97%)
Domains predictor:  Accuracy: 19372/20000 (97%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.343850, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.236943, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.216584, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.133211, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.202780, lr: 0.00273

In [0]:
grad_log = []
def grand(x, lambd):
  return backward_functions.GRand(1e-3, lambd, grad_log)(x)

In [89]:
test_grad_function(grand)

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1853807358>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f185046d3c8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", 

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.317810, lr: 0.01000 lambd: 0.00000


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1853738c50>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._

Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.712805, lr: 0.00726 lambd: 0.26052
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.296397, lr: 0.00580 lambd: 0.48792
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.255301, lr: 0.00488 lambd: 0.66404
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.488634, lr: 0.00425 lambd: 0.78820

Target Domain Test set: Average loss: 1.4624, Accuracy: 5065/10000 (51%)
Source Domain Test set: Average loss: 0.1266, Accuracy: 9597/10000 (96%)
Domains predictor:  Accuracy: 19248/20000 (96%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.316768, lr: 0.00391 lambd: 0.84828
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.188924, lr: 0.00351 lambd: 0.90812
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.195166, lr: 0.00320 lambd: 0.94505
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.286140, lr: 0.00295 lambd: 0.96740
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.112674, lr: 0.00273 lambd: 0.98074

Target Domain Test set: Average loss: 1.5594, Accuracy: 4544/

# Gradient Inverse Layer test

In [0]:
def grinv(x, lambd):
  return backward_functions.GRInv(-1, lambd)(x)

In [0]:
model_f = models.MnistFeatureExtractor().to(device)
model_c = models.MnistClassPredictor().to(device)
model_d = models.MnistDomain(grad_func=grinv).to(device)
optim_f = optim.SGD(model_f.parameters(), lr=lr, momentum=momentum, nesterov=True)
optim_c = optim.SGD(model_c.parameters(), lr=lr, momentum=momentum, nesterov=True)
optim_d = optim.SGD(model_d.parameters(), lr=lr, momentum=momentum, nesterov=True)

train_models = Models(model_f, model_c, model_d)
optims = Optims(optim_f, optim_c, optim_d)

In [0]:
model_trainer = trainer.Trainer(train_models, optims, criterions, device,
                                tune_lr=True)

In [104]:
test_history = defaultdict(lambda:[])
model_trainer.train(epochs, loaders, test_history=test_history)

Train Epoch: 					1 [0/60000 (0%)]	Loss: 2.312509, lr: 0.01000 lambd: 0.00000
Train Epoch: 					1 [12800/60000 (21%)]	Loss: 0.470715, lr: 0.00865 lambd: 0.10626
Train Epoch: 					1 [25600/60000 (43%)]	Loss: 0.401973, lr: 0.00766 lambd: 0.21015
Train Epoch: 					1 [38400/60000 (64%)]	Loss: 0.208745, lr: 0.00690 lambd: 0.30951
Train Epoch: 					1 [51200/60000 (85%)]	Loss: 0.331025, lr: 0.00630 lambd: 0.40253

Target Domain Test set: Average loss: 1.2261, Accuracy: 6299/10000 (63%)
Source Domain Test set: Average loss: 0.1081, Accuracy: 9680/10000 (97%)
Domains predictor:  Accuracy: 14677/20000 (73%)

Train Epoch: 					2 [0/60000 (0%)]	Loss: 0.371136, lr: 0.00595 lambd: 0.46212
Train Epoch: 					2 [12800/60000 (21%)]	Loss: 0.307323, lr: 0.00551 lambd: 0.54178
Train Epoch: 					2 [25600/60000 (43%)]	Loss: 0.395909, lr: 0.00514 lambd: 0.61276
Train Epoch: 					2 [38400/60000 (64%)]	Loss: 0.271887, lr: 0.00483 lambd: 0.67507
Train Epoch: 					2 [51200/60000 (85%)]	Loss: 0.285515, lr: 0.00455